In [ ]:
pip install xlsxwriter

In [ ]:
pip install openpyxl

In [ ]:
pip install -U sentence-transformers


In [ ]:
import sys
sys.path.append('../input')
from xuexibao.utils.fileUtils import *

In [ ]:
lines = readExcelToList('../input/datafc2/Facebook__20210101_20220322.xlsx')

In [ ]:
len(lines)

In [ ]:
lines[0]

In [ ]:
lines[1]

In [ ]:
lines[-1]

In [ ]:
for i in range(len(lines)):
    title = lines[i][3].strip('').strip('.').replace('!','').replace('\n','')+'. '
    lines[i][4] = title + lines[i][4]

In [ ]:
lines[-1]

In [ ]:
for i in range(len(lines)):
    lines[i][4] = lines[i][4].strip().replace('!','').replace('\n','').replace('-----------------------------------------------------------------------------------------','').replace('\t','')

In [ ]:
len(lines)

In [ ]:
def f1(t):
    temp = ''
    for i in t:
        if ord(i)<1200:
            temp+=i
    return temp

In [ ]:
for i in range(1,len(lines)):
    lines[i][4] = f1(lines[i][4])
    lines[i][3] = f1(lines[i][3])

review = {
    time:xxx,
    title:xxx,
    rating:xxx,
    reviewer:xxx,
    text:xxx,
    is_deleted:True,
    dev_reply:xxx
}

In [ ]:
arr = []
for i in lines:
    review = {
    'time':i[0],
    'title':i[3].strip().replace('!','').strip('.').replace('\n','').replace('\t','').replace('"',''),
    'rating':i[2],
    'reviewer':i[1],
    'text':i[4],
    'is_deleted':None,
    'dev_reply':None
}
    arr.append(review)
    


In [ ]:
# docx = list(set([i['text'], for i in arr]))

In [ ]:
docx = [i['text'] for i in arr]
time = [i['time'] for i in arr]
rating = [i['rating'] for i in arr]
title = [i['title'] for i in arr]

In [ ]:
list(set(rating))

In [ ]:
len(docx)

In [ ]:
f1 = readExcelToList('../input/facebook/f1.xlsx')

In [ ]:
f1[-10:]

In [ ]:
def ty_comment(name):
    s_comment = ''
    for i,j in f1:
        if j == name:
            s_comment+=i
    return s_comment


In [ ]:
comment = ty_comment('Disclosure')+ty_comment('Usage')+ty_comment('Collection')+ty_comment('Retention')

In [ ]:
comment = comment.replace('\xa0','').replace('\n','')

In [ ]:
comment

In [ ]:
from sentence_transformers import SentenceTransformer, util
#Load the model
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

In [ ]:
model
#0.472131

In [ ]:
def find_coment(query,docs,path):
    #Encode query and documents
    query_emb = model.encode(query)
    doc_emb = model.encode(docs)

    #Compute dot score between query and all document embeddings
    scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()
#     time = [i['time'] for i in arr]
#     rating = [i['rating'] for i in arr]
#     title = [i['title'] for i in arr]
    #Combine docs & scores
    doc_score_pairs = list(zip(docs, scores,title,rating,time))

    #Sort by decreasing score
    doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

    f = open(path,'w')
    f.write('id,socre,text,title,rating,time\n')
    count = 0
    for doc, score,a,b,c in doc_score_pairs:
        count+=1
        f.write('{},{},"{}","{}","{}","{}"\n'.format(count,score,doc,a,b,c))
        if score<0.472131:
            break

In [ ]:
find_coment(comment,docx,'./fc_book5.csv')